In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re

from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices

In [ ]:
def clean_date(bgs_index):
    if re.match(r'^\d{2}\s\w{3}\s+\d{4}$', bgs_index):
        return pd.to_datetime(bgs_index, format="%d %b %Y", errors='coerce').to_period('M').to_timestamp('M')
    else:
        return pd.to_datetime(bgs_index).to_period('M').to_timestamp('M')

def clean_percentage(x):
    if x.strip() in ['Variable','Floating']:
        return x
    try:
        x = float(x)
    except (ValueError, TypeError):
        units, fractions = x.split(' ')
        num, denom = map(float, fractions.split('/'))
        x = float(units) + num / denom
        print(x)
    return x

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')


In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)

In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

details.keys()

In [ ]:

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)


In [ ]:
pos = conv['31-03-2024':'30-04-2025'].sum(axis = 0).reset_index()
bonds_paying_interest=pos[pos[0]>0]['index'].to_list()


In [ ]:
gilt = bonds_paying_interest[0]
gilt

In [ ]:
conv['31-03-2024':'30-04-2025'][gilt]

In [ ]:
conv_details[conv_details['Sequence'] ==gilt]['%'].values[0]

In [ ]:
conv['31-03-2024':'30-04-2025'][gilt]*conv_details[conv_details['Sequence'] ==gilt]['%'].values[0]/12/100

In [ ]:
conv_interest = 0
for gilt in bonds_paying_interest:
    conv_interest += (conv['31-03-2024':'30-04-2025'][gilt]*conv_details[conv_details['Sequence'] ==gilt]['%'].values[0]/12/100).sum()
conv_interest

In [ ]:
tables.keys()

In [ ]:
linker_pos =  new_no_idx['31-03-2024':'30-04-2025'].sum(axis = 0).reset_index()
linkers_paying_interest=linker_pos[linker_pos[0]>0]['index'].to_list()
len(linkers_paying_interest)

In [ ]:
linker_inc = new_no_idx.loc['31-03-2024':'30-04-2025',linkers_paying_interest].diff().sum().reset_index().rename(columns={0:'Increase', 'index':'Sequence'})
linker_inc = linker_inc[linker_inc['Increase']>0].copy()

In [ ]:
linker_pos_check = new['31-03-2024':'30-04-2025'].sum(axis = 0).reset_index()
linker_check_list=linker_pos[linker_pos[0]>0]['index'].to_list()
[x for x in linker_check_list if x not in linkers_paying_interest]

In [ ]:
new_linker_interest = 0
for linker in linkers_paying_interest:
    new_linker_interest += (new['31-03-2024':'30-04-2025'][linker]*new_details[new_details['Sequence'] ==linker]['%'].values[0]/12/100).sum()
new_linker_interest

In [ ]:
linker_inc

In [ ]:
linkers_base = new['31-03-2024':'30-04-2025'].copy()
gross_change = linkers_base.diff().sum()

In [ ]:
start = '31-03-2024'
end = '30-04-2025'
idx_start = linkers_base.loc[start, linker_inc['Sequence'].to_list()]
idx_diff = linkers_base.loc[end, linker_inc['Sequence'].to_list()] - idx_start

In [ ]:
idx_diff

In [ ]:
idx_start

In [ ]:
no_idx_end = new_no_idx.loc[end, linker_inc['Sequence'].to_list()]
no_idx_diff = no_idx_end - new_no_idx.loc[start, linker_inc['Sequence'].to_list()]
no_idx_end

In [ ]:
(idx_diff + idx_start)/no_idx_end * no_idx_diff

In [ ]:
inflation_uplift = gross_change.sum() - ((idx_diff + idx_start)/no_idx_end * no_idx_diff).sum()

In [ ]:
old_linker_pos =  old[start:end].sum(axis = 0).reset_index()
old_linkers_paying_interest=old_linker_pos[old_linker_pos[0]>0]['index'].to_list()

In [ ]:
old.loc[start:end, old_linkers_paying_interest]

In [ ]:
old_linker_interest = 0
for linker in old_linkers_paying_interest:
    old_linker_interest += (old['31-03-2024':'30-04-2025'][linker]*old_details[old_details['Sequence'] ==linker]['%'].values[0]/12/100).sum()
old_linker_interest

In [ ]:
apf_losses = - 18500
conv_interest + new_linker_interest + old_linker_interest + inflation_uplift - apf_losses
# Actual Answer 105.2